In [ ]:
cfrom zipfile import ZipFile
file_name = '/content/output2.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import os
os.remove('/content/output2/PV/.DS_Store')
os.remove('/content/output2/PV/labels/.DS_Store')

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Load images and masks
def load_images_and_masks(image_dir, mask_dir, target_size=(512, 512)):
    images, masks = [], []
    for image_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, image_name)
        mask_path = os.path.join(mask_dir, image_name.replace('.tif', '_label.tif'))  # Assuming masks match image names
#         print(mask_path)
        if os.path.exists(mask_path):
            # Load and preprocess image and mask
#             print(mask_path)
            image = img_to_array(load_img(img_path, target_size=target_size))/255
            mask = img_to_array(load_img(mask_path, target_size=target_size, color_mode="grayscale"))/255
#             print(image)
            masks.append(mask)  # Keep as single-channel grayscale
            images.append(image)
    return np.array(images), np.array(masks)

# Load dataset
image_dir = r'/content/output2/PV'
mask_dir = r'/content/output2/PV/labels'
images, masks = load_images_and_masks(image_dir, mask_dir)

# Split into train and validation sets
# First, split into training (80%) and temp (20%) → temp will be further split
x_train, x_temp, y_train, y_temp = train_test_split(
    images, masks, test_size=0.2, random_state=42
)

# Now, split `x_temp` into validation (10%) and test (10%)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42
)

In [ ]:
import tensorflow.keras.backend as K

def iou_loss(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return 1 - iou  # Since we want to minimize loss, we use (1 - IoU)


In [ ]:
import tensorflow.keras.backend as K

def weighted_bce(y_true, y_pred, weight_background=0.2, weight_foreground=0.8):
    """Weighted Binary Cross-Entropy Loss"""
    bce = K.binary_crossentropy(y_true, y_pred)
    weights = y_true * weight_foreground + (1 - y_true) * weight_background
    return K.mean(bce * weights)


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate

def unet_model(input_size=(512, 512, 3)):
    inputs = Input(input_size)

    # Downsampling (Encoder)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Upsampling (Decoder)
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model

# Initialize model
unet = unet_model()
unet.compile(optimizer='adam', loss=weighted_bce, metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callbacks
checkpoint = ModelCheckpoint('unet_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = unet.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=75,
    batch_size=4,
    callbacks=[checkpoint]
)


Epoch 1/75
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8410 - loss: 0.1197   

In [ ]:
from google.colab import files
files.download("unet_model.keras")

In [ ]:
import matplotlib.pyplot as plt

# Plotx` accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
from posixpath import isabs
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


def calculate_iou(y_true, y_pred, threshold=0.5):
    """
    Calculate Intersection over Union (IoU) between true and predicted masks.

    Args:
    - y_true: Ground truth mask (numpy array or tensor of shape [H, W]).
    - y_pred: Predicted mask (numpy array or tensor of shape [H, W]).
    - threshold: Threshold to binarize the predicted mask (default: 0.5).

    Returns:
    - IoU score (float).
    """
    # Binarize the predicted mask
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    y_true = tf.cast(y_true > 0, tf.float32)  # Ensure ground truth is binary

    # Calculate intersection and union
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    # Avoid division by zero
    iou = intersection / (union + tf.keras.backend.epsilon())
    return iou.numpy()

# Example usage
# Assuming `original_mask` and `predicted_mask` are numpy arrays of shape [H, W]


# Predict on a single image
def predict_and_visualize(image_path, label_path, model):
    image = img_to_array(load_img(image_path, target_size=(512, 512))) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)[0, ..., 0]  # Remove batch and channel dimensions

    y_pred = tf.cast(prediction > 0.5, tf.float32)
    #loading true lable
    lab= img_to_array(load_img(label_path, target_size=(512, 512), color_mode="grayscale")) / 255.0
    # lab = np.expand_dims(image, axis=0)
    lab= tf.squeeze(lab)
    y_true=  tf.cast(lab> 0, tf.float32)

    # Calculate intersection and union
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    # Avoid division by zero
    iou = intersection / (union + tf.keras.backend.epsilon())
    print(iou.numpy())

    # Display original image and prediction
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(image[0])
    plt.axis('off')
    #Original mask
    plt.subplot(1, 2, 2)
    plt.title("Original Mask")
    plt.imshow(lab, cmap='gray')
    plt.axis('off')
    plt.show()


# Assuming `predicted_mask` is the grayscale mask
    threshold =0.5 # Adjust threshold as necessary
    binary_mask = (prediction>threshold ).astype(np.uint8)

    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask")
    plt.imshow(binary_mask, cmap='gray')
    plt.axis('off')
    plt.show()

# Example
predict_and_visualize('/content/output2/PV/r0493.tif','/content/output2/PV/labels/r0493_label.tif', unet)


In [ ]:
import numpy as np
import zipfile
import os

# Assuming x_train, y_train, x_val, y_val, x_test, y_test are numpy arrays
save_path = "/content/dataset.npz"  # Save in an accessible directory

# Save dataset in compressed format
np.savez_compressed(save_path,
                    x_train=x_train, y_train=y_train,
                    x_val=x_val, y_val=y_val,
                    x_test=x_test, y_test=y_test)

print(f"Dataset saved at: {save_path}")


In [ ]:
import tensorflow as tf
import numpy as np

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)  # Flatten to 1D
    y_pred_f = tf.keras.backend.flatten(y_pred)

    intersection = tf.reduce_sum(y_true_f * y_pred_f)  # Count overlapping pixels
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)

    return (2. * intersection + smooth) / (union + smooth)

In [ ]:
from posixpath import isabs
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array


#<--------------------------------------------------------------------------------------------------------------------------->
# Predict on a single image
def predict_and_visualize(image_path, label_path, model):
    image = img_to_array(load_img(image_path, target_size=(512, 512))) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)[0, ..., 0]  # Remove batch and channel dimensions

    y_pred = tf.cast(prediction > 0.5, tf.float32)
    # print("Predicted Label",y_pred)
    #loading true lable
    lab= img_to_array(load_img(label_path, target_size=(512, 512), color_mode="grayscale")) / 255.0
   # Remove extra dimension (512, 512, 1) → (512, 512)
    lab= tf.squeeze(lab)
    y_true= tf.cast(lab> 0, tf.float32)


    # Calculate intersection and union
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    # Avoid division by zero
    iou = intersection / (union + tf.keras.backend.epsilon())
    print("IoU Score :",iou.numpy())
    #<--------------------------------------------------------------------------------------------------------------------------->
    print("Dice Score: ", dice_coefficient(y_true, y_pred, smooth=1e-6))
    # Display original image and prediction
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(image[0])
    plt.axis('off')
    #Original mask
    plt.subplot(1, 2, 2)
    plt.title("Original Mask")
    plt.imshow(y_true, cmap='gray')
    plt.axis('off')
    plt.show()
    #<--------------------------------------------------------------------------------------------------------------------------->

# Assuming `predicted_mask` is the grayscale mask
    threshold =0.5# Adjust threshold as necessary
    binary_mask = (prediction > threshold).astype(np.uint8)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Predicted Mask (binary_mask)")
    plt.imshow(binary_mask, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask(Actual)")
    plt.imshow(prediction, cmap='gray')
    plt.axis('off')
    plt.show()
 #<--------------------------------------------------------------------------------------------------------------------------->

predict_and_visualize(r"/content/output2/PV/r0361.tif",r"/content/output2/PV/labels/r0361_label.tif", unet)


In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(x_test[20])
plt.axis('off')
#Original mask
plt.subplot(1, 2, 2)
plt.title("Original Mask")
plt.imshow(y_test[20], cmap='gray')
plt.axis('off')
plt.show()
#<--------------------------------------------------------------------------------------------------------------------------->
prediction = unet.predict(x_test[20:21])[0, ..., 0]
#<--------------------------------------------------------------------------------------------------------------------------->
# Assuming `predicted_mask` is the grayscale mask
threshold =0.5# Adjust threshold as necessary
binary_mask = (prediction > threshold).astype(np.uint8)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Predicted Mask (binary_mask)")
plt.imshow(binary_mask, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Predicted Mask(Actual)")
plt.imshow(prediction, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
def calculate_iou(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + tf.keras.backend.epsilon()) / (union + tf.keras.backend.epsilon())

# Function to calculate Dice Score
def calculate_dice(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + tf.keras.backend.epsilon()) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + tf.keras.backend.epsilon())

def accuracy(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Convert predictions to binary
    correct = tf.equal(y_true, y_pred)  # Compare predictions with ground truth
    return tf.reduce_mean(tf.cast(correct, tf.float32))

def recall(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)

    true_positives = tf.reduce_sum(y_true * y_pred)  # Count correctly predicted foreground pixels
    possible_positives = tf.reduce_sum(y_true)  # Total foreground pixels in ground truth
    return true_positives / (possible_positives + tf.keras.backend.epsilon())


def precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)

    true_positives = tf.reduce_sum(y_true * y_pred)  # Count correctly predicted foreground pixels
    predicted_positives = tf.reduce_sum(y_pred)  # Total predicted foreground pixels
    return true_positives / (predicted_positives + tf.keras.backend.epsilon())


def evaluate_dataset(model, X, y):
    """ Evaluate Accuracy, IoU, Precision, Recall, and Dice Score for a dataset. """
    iou_scores, dice_scores, accuracies, precisions, recalls = [], [], [], [], []
    count=0

    for i in range(len(X)):
        image = np.expand_dims(X[i], axis=0)  # Add batch dimension
        true_mask = y[i]
        true_mask= tf.squeeze(true_mask)
        true_mask= tf.cast(true_mask> 0, tf.float32)

        # Get model prediction
        pred_mask = model.predict(image, verbose=0)[0, ..., 0]  # Remove batch dim
        pred_mask =  tf.cast(pred_mask> 0.5, tf.float32)  # Convert to binary mask

        # Calculate metrics
        iou_scores.append(calculate_iou(true_mask, pred_mask))
        dice_scores.append(calculate_dice(true_mask, pred_mask))
        accuracies.append(accuracy(true_mask, pred_mask))
        precisions.append(precision(true_mask, pred_mask))
        recalls.append(recall(true_mask, pred_mask))
        count=count+1
        print(count)


    # Compute mean scores
    return {
        "Mean Accuracy": np.mean(accuracies),
        "Mean IoU": np.mean(iou_scores),
        "Mean Dice Score": np.mean(dice_scores),
        "Mean Precision": np.mean(precisions),
        "Mean Recall": np.mean(recalls),
    }


In [ ]:
train_metrics = evaluate_dataset(unet, x_train, y_train)
val_metrics = evaluate_dataset(unet, x_val, y_val)
test_metrics = evaluate_dataset(unet, x_test, y_test)

print("Train Metrics:", train_metrics)
print("Validation Metrics:", val_metrics)
print("Test Metrics:", test_metrics)


In [ ]:
from google.colab import files
files.download("unet_model.keras")